In [1]:
import sys
import numpy as np
import pandas as pd
import os
sys.path.append('..')
from model.bert.bert_textdataset import BertTextDataset
from model.bert.bert_classfier import BertFCClassifier
from model.bert.bert_embedder import BertEmbedder
from model.bert.bert_evaluator import BertFCEvaluator
from model.bert.bert_trainer import BertFCTrainer
from nlp_pipeline.preprocess_text import TextPreprocessor
from sklearn.model_selection import train_test_split
from nlp_pipeline.back_translator import BackTranslationAugmentor

/hop/home/muhammad_mufli_ramadhan/tfid/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/features/1.csv', encoding='latin1')
df = df[['transcription', 'label']]
df.dropna(inplace=True)

df = df[df['label'].isin(['ASD', 'NON ASD'])]

AUGMENTED DATA. DO THIS IF NECESSARY

In [2]:
df = pd.read_csv('../data/features/1.csv', encoding='latin1')
columns = ['label', 'transcription', 'total_words', 'unique_words', 'num_sentences',
                   'stopwords', 'num_adjectives', 'num_nouns', 'num_verbs', 'num_adverbs',
                   'type_token_ratio', 'avg_words_per_sentence']
df = df[columns]
augmentor = BackTranslationAugmentor()
df_augmented = augmentor.augment_dataframe(df)
print(df_augmented.head)
df_augmented.to_csv('../data/features/1_augmented.csv', index=False)

/hop/home/muhammad_mufli_ramadhan/tfid/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Processing 2324 utterances for back translation...


  0%|          | 0/2324 [00:00<?, ?it/s]/hop/home/muhammad_mufli_ramadhan/tfid/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4240: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
100%|██████████| 2324/2324 [14:13<00:00,  2.72it/s]  

<bound method NDFrame.head of         label                   transcription  total_words  unique_words  \
0     NON ASD                Tunjuk kaka coba            3             3   
1         ASD                           Inii!            2             2   
2     NON ASD                   Siapa namamu?            4             4   
3         ASD                         Iyaaaa?            2             2   
4     NON ASD                  Kenalan duluu!            3             3   
...       ...                             ...          ...           ...   
5887  NON ASD                            Oke?            2             2   
5888  NON ASD       Mari kita pergi ke bunga.            6             6   
5889  NON ASD  Seperti Anda sudah cukup, ibu.            7             7   
5890  NON ASD                Kiana lihat kaka            3             3   
5891  NON ASD                Lihatlah saudara            3             3   

      num_sentences  stopwords  num_adjectives  num_nouns

In [ ]:
text_preprocessor = TextPreprocessor()
df['clean_text'] = df['transcription'].apply(text_preprocessor.preprocess)

In [ ]:
embedder = BertEmbedder()
tokenizer = embedder.tokenizer
dataset = BertTextDataset(df=df[['clean_text', 'label']], tokenizer=tokenizer)
y = (df['label']== 'ASD').astype(int).values
X = embedder.encode_series(df['clean_text'].tolist())

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
model = BertFCClassifier(input_dim=768, hidden_dim=128, dropout=0.3)

trainer = BertFCTrainer(
    model=model,
    lr=2e-5,
    batch_size=16,
    epochs=20,
    patience=3
)
print (X_train.shape, y_train.shape)

(2854, 768) (2854,)


In [ ]:
trainer.train(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val
)
trainer.evaluate(
    X_test=X_val,
    y_test=y_val
)

sudah di update
Epoch 1/20, Train Loss: 71.0762, Val Loss: 0.4246
Epoch 2/20, Train Loss: 70.6640, Val Loss: 0.4212
Epoch 3/20, Train Loss: 70.2911, Val Loss: 0.4217
Epoch 4/20, Train Loss: 70.0362, Val Loss: 0.4201
Epoch 5/20, Train Loss: 70.0063, Val Loss: 0.4193
Epoch 6/20, Train Loss: 69.6845, Val Loss: 0.4197
Epoch 7/20, Train Loss: 69.2611, Val Loss: 0.4184
Epoch 8/20, Train Loss: 68.5228, Val Loss: 0.4181
Epoch 9/20, Train Loss: 68.5416, Val Loss: 0.4176
Epoch 10/20, Train Loss: 68.2078, Val Loss: 0.4171
Epoch 11/20, Train Loss: 68.1157, Val Loss: 0.4166
Epoch 12/20, Train Loss: 67.6746, Val Loss: 0.4169
Epoch 13/20, Train Loss: 67.2832, Val Loss: 0.4162
Epoch 14/20, Train Loss: 67.3722, Val Loss: 0.4159
Epoch 15/20, Train Loss: 66.8521, Val Loss: 0.4156
Epoch 16/20, Train Loss: 67.0199, Val Loss: 0.4157
Epoch 17/20, Train Loss: 66.1453, Val Loss: 0.4149
Epoch 18/20, Train Loss: 67.1085, Val Loss: 0.4152
Epoch 19/20, Train Loss: 65.3855, Val Loss: 0.4146
Epoch 20/20, Train Loss: